# Descarga datos esios

## Documentacion:

La nueva web pública de e·sios pone a disposición de todos los usuarios una API para la descarga de información, cuya documentación se encuentra disponible en https://api.esios.ree.es.

Para poder utilizar esta API deberán solicitar un token personal enviando un correo a **consultasios@ree.es**.En mi caso:

token: 9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10

En nuestro caso, la solicitud de datos será para un periodo de tiempo solicitando los datos por código indicador:

https://api.esios.ree.es/indicator/getting_a_specific_indicator_filtering_values_by_a_date_range


## 1. Importamos librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [77]:
pd.options.display.max_columns= None

In [78]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      /anaconda3/lib/python3.6/site-packages/requests/api.py
Type:      function


## 2. Definición features

Creamos diccionario

In [79]:
d={10010: 'P48_EOLICA',
   10027: 'P48_DEMANDA',
   10008: 'P48_CARBON',
   475: 'POT_DISP_HULLA_ANT',  #Los tendremos que sumar para obtener MW disponibles Carbón
   476: 'POT_DISP_HULA_SB',
   600: [['PRECIO_MD_ESP',3],['PRECIO_MD_FR',2]],
   612: [['PRECIO_I1',3]]
  }

In [80]:
d.items()

dict_items([(10010, 'P48_EOLICA'), (10027, 'P48_DEMANDA'), (10008, 'P48_CARBON'), (475, 'POT_DISP_HULLA_ANT'), (476, 'POT_DISP_HULA_SB'), (600, [['PRECIO_MD_ESP', 3], ['PRECIO_MD_FR', 2]]), (612, [['PRECIO_I1', 3]])])

En el caso del id_precios habrá que adentrarse un nivel adicional y consultar GEOIDS:

* ESPAÑA = 3
* FRANCIA = 2

## 3. Funciones solicitud datos

Función para obtener los datos del indicador *ind* solicitado entre dos fechas dadas.

En nuestro dataset se nos presentan 2 peculiaridades a introducir en la función:

1) Existe un único indicador para los precios de cada mercado, diferenciándose el pais por el parámetro GEOIDS --> **solicita_datos_precios**

2) Los datos de potencia disponible hay que agregarlos porque viene desglosada en comunidades --> **solicita datos disponibles**

In [81]:
def solicita_datos_general(fecha_ini,fecha_fin,ind):

    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }

    query_parametros_={'start_date': fecha_ini,
                'end_date':fecha_fin}

    url_='https://api.esios.ree.es/indicators/' + str(ind)
    
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
    
    #Comprobamos que trae datos
    assert r.status_code==200
    
    #r.content --> durante el diseño para explorar datos
    
    data=json.loads(r.content)
    
    #type(data)
    #data['indicator']['values'] --> durante el diseño para explorar datos
    
    df_data=pd.DataFrame(data['indicator']['values'])
    
    #df_data.dtypes --> Tenemos que convertir las fechas a formato fecha
    #y crear nuestro df
    
    df=pd.DataFrame()        
    df['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data['datetime']]) 
    df['value']=df_data['value']
    if (ind==475) | (ind==476):
        df=df.groupby('datetime')['value'].sum()
        df=pd.DataFrame(df)
    else:
        df=df.set_index('datetime')

    return df

In [82]:
def solicita_datos_precios(fecha_ini,fecha_fin,ind,lista):
   
    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }
 
    query_parametros_={'start_date': fecha_ini,
                       'end_date': fecha_fin}
 
    url_='https://api.esios.ree.es/indicators/'+ str(ind)
 
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
 
    #Comprobamos que trae datos
    assert r.status_code==200
 
    data=json.loads(r.content)
 
    data['indicator']['values']
 
    df_data=pd.DataFrame(data['indicator']['values'])
 
    df=pd.DataFrame()
    for i in enumerate(lista):
        df_subset=pd.DataFrame()
        df_data_subset=df_data[df_data['geo_id']==i[1][1]]
        df_data_subset.reset_index(drop=True,inplace=True)
        df_subset['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data_subset['datetime']])
        df_subset[str(i[1][0])]=df_data_subset['value']
        df_subset.set_index('datetime',inplace=True)
        df=pd.concat([df,df_subset],axis=1)
    return df      
 

## 4. Agregación dataset

In [86]:
datos_esios=pd.DataFrame()  #será nuestro set de datos

fecha_ini_='2015-10-01 00:00:00'
fecha_fin_='2018-09-30 23:59:59'

Definimos parámetros de entrada:

La idea inicial es que mi dataset empiece en 01/10/2015 hasta 30/09/2018.

He escogido 3 años móviles, teniendo en cuenta que el año hidráulico comienza el octubre (y la hidraulicidad es importante en el Mercado eléctrico) e intentar evitar el efecto de cambios regulatorios en la mayor medida posible

In [87]:
for k,v in d.items():
    if (k==600) | (k==612):
        historico=solicita_datos_precios(fecha_ini_,fecha_fin_,k,v)
    else:    
        historico=solicita_datos_general(fecha_ini_,fecha_fin_,k)
        historico.columns=[str(v)]
    datos_esios=pd.concat([datos_esios,historico],axis=1)
    

In [88]:
datos_esios

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1
datetime,,,,,,,,
2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31
2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25
2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69
2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10
2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25
2015-10-01 05:00:00,1327.0,22183.9,8223.3,5355.4,4705.3,53.60,32.87,53.74
2015-10-01 06:00:00,1347.4,24139.8,8014.0,5355.4,4705.3,58.02,44.97,58.02
2015-10-01 07:00:00,1363.3,28141.4,8102.1,5355.4,4705.3,63.03,56.00,62.82
2015-10-01 08:00:00,1364.5,29311.3,8079.3,5355.4,4705.3,63.56,59.06,63.56


## 5. Check datos

In [27]:
datos_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,3142.968750,25155.120833,5015.925000,69.637708,57.886250,69.936250
std,1504.425027,2608.289459,680.723847,4.644057,8.929302,5.788725
min,849.200000,20378.700000,3655.500000,61.030000,41.450000,60.170000
25%,2042.825000,22960.625000,4541.025000,65.100000,51.735000,64.615000
50%,2858.000000,25281.300000,4914.050000,70.340000,57.295000,70.285000
75%,4097.125000,26973.300000,5692.550000,73.445000,63.562500,75.032500
max,6618.700000,29449.000000,6027.600000,77.490000,77.490000,78.690000


In [28]:
datos_esios.shape  #para testear que no tengo NA

(48, 6)

## 6. Salvar datos